<a href="https://colab.research.google.com/github/YG15/DataHack2019/blob/master/model_train_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sklearn as sk
import matplotlib.pyplot as plt
import json
from urllib import request
import random
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

In [3]:
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
url = 'https://armis-datahack.s3.amazonaws.com/all_devices.csv'
devices_df = pd.read_csv(url, index_col = 0)
#devices_df.head()

In [0]:
final_df = pd.read_csv(r'drive/My Drive/only_session_Features_no_log.csv', index_col =0)

In [0]:
# final_df = final_df.replace([np.inf, -np.inf], np.nan)
# final_df = final_df.fillna(0)

In [0]:
from sklearn.mixture import GaussianMixture
from pyod.models.knn import KNN

def run_IF(df,ms=100, rs=1):
    model = IsolationForest(behaviour = 'new', max_samples=ms, 
                          random_state = rs, contamination = 0)
    model.fit(df)
    anomaly_scores = model.decision_function(df)
    
    return anomaly_scores

def run_KNN(df):
  
    clf = KNN()
    clf.fit(X)
    # get outlier scores
    anomaly_scores = clf.decision_scores_ 
    return anomaly_scores

In [0]:
#find most varied features
variety_df = pd.DataFrame(columns=['feature', 'max','min','std'],)
for i,c in enumerate(df_gb.columns):
  df2 =pd.DataFrame({'feature':c,'max':df_gb[c].max(),'min': df_gb[c].min(),'std':df_gb[c].std()}, index=[0])
  variety_df = variety_df.append(df2, ignore_index=True)

In [0]:
variety_df.sort_values('std', ascending=False)['feature'].values

In [0]:
import random
iterations =1000
scores_list =[]
selected_col = ['network_id',
                'device_id',
                'type_ip_camera',
                'type_mobile_phone',
                'type_pc',
                'type_printer',
                'type_tablet',
                'type_voip',
                'type_watch',
                'host_ipnunique',
                'port_dstnunique',
                'timestampnunique',
                'timestampcount',
                'timestamprange',
                'transport_protocolnunique',
                'packets_countnanmax',
                'packets_countnanmean',
                'packets_countnanstd',
                'outbound_bytes_countnanmax',
                'outbound_bytes_countnanmean',
                'inbound_bytes_countnanmax',
                'inbound_bytes_countnanmean',
                'retransmit_countnanmax',
                'retransmit_countnanmean',
                'retransmit_countnanstd',
                'latencynanmax']
test_df = final_df[selected_col].fillna(final_df.mean())

for i in range(iterations):
    col_number =random.randrange(len(test_df.columns)+1)
    while col_number<3:
      col_number = random.randrange(len(test_df.columns)+1)
    print(i, col_number)
    selected_col = list(set(random.sample(set(test_df.columns), col_number)+['network_id','device_id']))
    df_features = test_df[selected_col]

    X = df_features.copy()
    X.drop(['device_id'], inplace=True, axis=1)

    # Run model
    sub_anomaly_scores = run_IF(X)
    sub_anomaly_scores = -sub_anomaly_scores
    scores_list.append(sub_anomaly_scores)

anomaly_scores =list(np.mean(scores_list, axis=0))
# Save results
results = pd.DataFrame({'network_id' : df_features.network_id, 'device_id' : df_features.device_id,'confidence' : list(anomaly_scores)})

In [175]:
results = pd.DataFrame({'network_id' : df_features.network_id, 'device_id' : df_features.device_id,'confidence' : features_set[4]['prediction']})
display(results.head())
print(results.shape)

,network_id,device_id,confidence
0,0,1028623,-0.422530
1,0,48047,-0.421168
2,0,123568,-0.420188
3,0,95366,-0.420829
4,0,1755023,-0.293364


(76038, 3)


In [176]:
#%% Submission

arr_to_submit = results.to_json(orient='values')

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "The Whale and the Petunias"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))

{'member': 'The Whale and the Petunias', 'rank': 4, 'score': 0.8657974757950241}


In [124]:
import random
iterations =10
features_set ={}

selected_col = ['network_id',
                'device_id',
                'type_ip_camera',
                'type_mobile_phone',
                'type_pc',
                'type_printer',
                'type_tablet',
                'type_voip',
                'type_watch',
                'host_ipnunique',
                'port_dstnunique',
                'timestampnunique',
                'timestampcount',
                'timestamprange',
                'transport_protocolnunique',
                'packets_countnanmax',
                'packets_countnanmean',
                'packets_countnanstd',
                'outbound_bytes_countnanmax',
                'outbound_bytes_countnanmean',
                'inbound_bytes_countnanmax',
                'inbound_bytes_countnanmean',
                'retransmit_countnanmax',
                'retransmit_countnanmean',
                'retransmit_countnanstd',
                'latencynanmax']
test_df = final_df[selected_col].fillna(final_df.mean())
for i in range(iterations):
    features_set[i]={}
    col_number = random.randrange(len(test_df.columns)+1)
    print('Iteration {}, {} features'.format(i, col_number))
    selected_col = list(set(random.sample(set(test_df.columns), col_number)+['network_id','device_id']))
    features_set[i]['features'] = selected_col
    df_features = test_df[selected_col]

    X = df_features.copy()
    X.drop(['device_id'], inplace=True, axis=1)

    # Run model
    anomaly_scores = run_IF(X)
    anomaly_scores = -anomaly_scores
    features_set[i]['prediction'] =anomaly_scores

    results = pd.DataFrame({'network_id' : df_features.network_id, 'device_id' : df_features.device_id,'confidence' : list(anomaly_scores)})
    arr_to_submit = results.to_json(orient='values')

    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = "The Whale and the Petunias"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)
    print(json.load(resp))
    resp = request.urlopen(req)
    score=(json.load(resp))
    features_set[i]['score'] = score

Iteration 0, 4 features
{'member': 'The Whale and the Petunias_draft_0', 'rank': 18, 'score': 0.830088057499539}
Iteration 1, 14 features
{'member': 'The Whale and the Petunias_draft_1', 'rank': 18, 'score': 0.8207414773975654}
Iteration 2, 11 features
{'member': 'The Whale and the Petunias_draft_2', 'rank': 17, 'score': 0.83044381902755}
Iteration 3, 19 features
{'member': 'The Whale and the Petunias_draft_3', 'rank': 20, 'score': 0.8143113512185732}
Iteration 4, 18 features
{'member': 'The Whale and the Petunias_draft_4', 'rank': 8, 'score': 0.8657974757950241}
Iteration 5, 11 features
{'member': 'The Whale and the Petunias_draft_5', 'rank': 10, 'score': 0.8526342647844835}
Iteration 6, 11 features
{'member': 'The Whale and the Petunias_draft_6', 'rank': 21, 'score': 0.8152956565722124}
Iteration 7, 0 features
{'member': 'The Whale and the Petunias_draft_7', 'rank': 58, 'score': 0.571054812763187}
Iteration 8, 2 features
{'member': 'The Whale and the Petunias_draft_8', 'rank': 21, 's

In [0]:
features_set[5]['features']

In [197]:
selected_col = ['network_id',
                'device_id',
                'host_ipnunique',
                'outbound_bytes_countnanmax',
                'timestampcount',
                'timestampnunique',
                'timestamprange',
                'transport_protocolnunique',
                'port_dstnunique']
df_features = final_df[selected_col].fillna(final_df.mean())

X = df_features.copy()
X.drop(['device_id'], inplace=True, axis=1)

# Run model
anomaly_scores = run_IF(X,ms=1024, rs=42)
anomaly_scores = -anomaly_scores

results = pd.DataFrame({'network_id' : df_features.network_id, 'device_id' : df_features.device_id,'confidence' : anomaly_scores})
arr_to_submit = results.to_json(orient='values')

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "The Whale and the Petunias"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))

{'member': 'The Whale and the Petunias', 'rank': 4, 'score': 0.8753833638145928}


In [196]:
elected_col = ['network_id',
                'device_id',
                'host_ipnunique',
                'outbound_bytes_countnanmax',
                'timestampcount',
                'timestampnunique',
                'timestamprange',
                'transport_protocolnunique',
                'port_dstnunique']
df_features = final_df[selected_col].fillna(final_df.mean())

X = df_features.copy()
X.drop(['device_id'], inplace=True, axis=1)

for i in range(15):
    print(i)
    # Run model
    sub_anomaly_scores = run_IF(X,ms=1024, rs=random.randint(0,5000))
    sub_anomaly_scores = -sub_anomaly_scores
    scores_list.append(sub_anomaly_scores)

anomaly_scores =list(np.mean(scores_list, axis=0))
# Save results
results = pd.DataFrame({'network_id' : df_features.network_id, 'device_id' : df_features.device_id,'confidence' : anomaly_scores})
#%% Submission

arr_to_submit = results.to_json(orient='values')

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "The Whale and the Petunias"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
{'member': 'The Whale and the Petunias', 'rank': 6, 'score': 0.8465436628183733}


In [0]:
import pandas as pd
import numpy as np


def get_network_hierarchy(network_df):
  """
  input: a table of network connection
  output: the same table with additional columns representing the level of hierarchy in the net
  IMPORTANT: the function assumes there each device has it unique id regardless the network it is in, if there are
  duplicates ids, the function will results incorrect results
            thus it is recommended to run the function on each network separately or to change the devices ids so
            they won't be any shared ids across networks
  """
  # Correct columns values from float tp integers
  network_df['full_service_device_id'] = [str(int(network))+'_'+str(int(device)) if ~np.isnan(device) else None\
                                          for network,device in zip(network_df.network_id,network_df.service_device_id)]
  network_df['full_device_id']         = [str(int(network))+'_'+str(int(device)) if ~np.isnan(device) else None\
                                          for network,device in zip(network_df.network_id,network_df.device_id)]
  
  output_devices = list(network_df.full_device_id.unique())
  input_devices  = list(network_df.full_service_device_id.unique())
  root_devices   = list(set(output_devices).difference(set(input_devices)))
  
  device_level = 0
  network_df['network_hierarchy'] = np.nan
  
  while len(root_devices)>0:
    network_df['network_hierarchy'] = [device_level+1 if device in root_devices else val for device, val in zip(network_df.full_device_id,network_df.network_hierarchy)]
    root_devices = list(network_df.full_service_device_id[network_df.network_hierarchy==device_level+1])
    device_level+=1
    print(device_level)
  return(network_df)

In [158]:
url = 'https://armis-datahack.s3.amazonaws.com/all_sessions.csv'
sessions_df = pd.read_csv(url,usecols = ['network_id','device_id','service_device_id'])
sessions_df.head()

,network_id,device_id,service_device_id
0,0,35,790889.0
1,0,35,1604622.0
2,0,57,NaN
3,0,57,NaN
4,0,57,NaN


In [0]:
network_df = get_network_hierarchy(sessions_df)